In [3]:
import pandas as pd

# Load the dataset (you can adjust the file path)
file_path = 'Computer_Assisted_Mass_Appraisal_-_Residential.csv'
dataset = pd.read_csv(file_path)

# Drop columns with high percentage of missing values (like 'YR_RMDL')
dataset_cleaned = dataset.drop(columns=['YR_RMDL'])

# For numerical columns, fill missing values with the median (e.g., for 'PRICE', 'ROOMS', etc.)
dataset_cleaned['PRICE'].fillna(dataset_cleaned['PRICE'].median(), inplace=True)
dataset_cleaned['ROOMS'].fillna(dataset_cleaned['ROOMS'].median(), inplace=True)
dataset_cleaned['BEDRM'].fillna(dataset_cleaned['BEDRM'].median(), inplace=True)
dataset_cleaned['KITCHENS'].fillna(dataset_cleaned['KITCHENS'].median(), inplace=True)
dataset_cleaned['FIREPLACES'].fillna(dataset_cleaned['FIREPLACES'].median(), inplace=True)

# For categorical columns, fill missing values with the mode (e.g., for 'STYLE', 'GRADE', etc.)
dataset_cleaned['STYLE'].fillna(dataset_cleaned['STYLE'].mode()[0], inplace=True)
dataset_cleaned['GRADE'].fillna(dataset_cleaned['GRADE'].mode()[0], inplace=True)
dataset_cleaned['STRUCT'].fillna(dataset_cleaned['STRUCT'].mode()[0], inplace=True)

# You can continue this process for other columns as needed

# Save the cleaned dataset if needed
dataset_cleaned.to_csv('cleaned_dataset.csv', index=False)

# Check if there are any remaining missing values
print(dataset_cleaned.isnull().sum())

SSL                     0
BATHRM               1807
HF_BATHRM            1807
HEAT                 1807
HEAT_D               1807
AC                   1807
NUM_UNITS            1807
ROOMS                   0
BEDRM                   0
AYB                    24
EYB                     0
STORIES              1975
SALEDATE                0
PRICE                   0
QUALIFIED               0
SALE_NUM                0
GBA                     0
BLDG_NUM                0
STYLE                   0
STYLE_D              1807
STRUCT                  0
STRUCT_D             1807
GRADE                   0
GRADE_D              1807
CNDTN                1807
CNDTN_D              1807
EXTWALL              1807
EXTWALL_D            1807
ROOF                 1807
ROOF_D               1807
INTWALL              1807
INTWALL_D            1807
KITCHENS                0
FIREPLACES              0
USECODE                 0
LANDAREA                0
GIS_LAST_MOD_DTTM       0
OBJECTID                0
dtype: int64